In [ ]:
import osimport sysimport jsonimport threadingfrom pathlib import Pathfrom datetime import datetimeimport tkinter as tkfrom tkinter import ttk, filedialog, messagebox, scrolledtextfrom collections import Counterimport reimport customtkinter as ctkimport whisperimport sounddevice as sdimport soundfile as sfimport numpy as npfrom datetime import datetimefrom anthropic import Anthropicfrom typing import Optional, Dict, Any, List, Tupleimport queueimport timeimport subprocessimport tempfile# Set appearance mode and color themectk.set_appearance_mode("dark")ctk.set_default_color_theme("blue")class AudioAnalyzerApp(ctk.CTk):    def __init__(self):        super().__init__()                self.title("Audio Transcription & Analysis Tool")        self.geometry("1200x800")                # Initialize variables        self.audio_file_path = None        self.transcribed_text = ""        self.api_key = None        self.whisper_model = None        self.error_queue = queue.Queue()        self.model_loading = False                # History for processed results (prompt, output) pairs        self.process_history: List[Tuple[str, str]] = []        self.current_history_index = -1                # Recording variables (in __init__)        self.is_recording = False        self.recording_data = []        self.recording_samplerate = 44100        self.recording_thread = None        self.recording_start_time = None        self.recorded_file_path = None        self.recording_process = None  # Initialize as None                # Configure grid weight        self.grid_columnconfigure(0, weight=1)        self.grid_rowconfigure(0, weight=1)                # Create main container        self.main_container = ctk.CTkFrame(self)        self.main_container.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)        self.main_container.grid_columnconfigure(1, weight=1)        self.main_container.grid_rowconfigure(0, weight=1)                # Create UI components        self.create_sidebar()        self.create_main_panel()                # Start checking for errors from background threads        self.check_error_queue()                # Initialize Whisper model in background        self.load_whisper_model()            def create_sidebar(self):        # Create a scrollable sidebar container        sidebar_container = ctk.CTkFrame(self.main_container, width=320)        sidebar_container.grid(row=0, column=0, sticky="nsew", padx=(0, 10))        sidebar_container.grid_propagate(False)        sidebar_container.grid_rowconfigure(0, weight=1)        sidebar_container.grid_columnconfigure(0, weight=1)                # Create scrollable frame        self.sidebar_scroll = ctk.CTkScrollableFrame(            sidebar_container,            width=300,            corner_radius=0        )        self.sidebar_scroll.grid(row=0, column=0, sticky="nsew")                # Use self.sidebar_scroll as the parent for all sidebar content        self.sidebar = self.sidebar_scroll  # For compatibility with existing code                # Title        title_label = ctk.CTkLabel(            self.sidebar,             text="Audio Analyzer",             font=ctk.CTkFont(size=24, weight="bold")        )        title_label.pack(pady=(20, 20))                # Step 1: File Selection Section        step1_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")        step1_frame.pack(fill="x", padx=20, pady=(0, 15))                ctk.CTkLabel(            step1_frame,             text="Step 1: Select Audio File",            font=ctk.CTkFont(size=14, weight="bold")        ).pack(anchor="w", pady=(0, 10))                self.file_button = ctk.CTkButton(            step1_frame,            text="📁 Choose Audio File",            command=self.select_audio_file,            height=40        )        self.file_button.pack(fill="x", pady=(0, 5))                self.file_label = ctk.CTkLabel(            step1_frame,             text="No file selected",             wraplength=250,            font=ctk.CTkFont(size=11)        )        self.file_label.pack(anchor="w")                # Recording Section (between Step 1 and Step 2)        recording_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")        recording_frame.pack(fill="x", padx=20, pady=(0, 15))                ctk.CTkLabel(            recording_frame,            text="Or Record Audio:",            font=ctk.CTkFont(size=14, weight="bold")        ).pack(anchor="w", pady=(0, 10))                # Recording controls frame        record_controls = ctk.CTkFrame(recording_frame)        record_controls.pack(fill="x")                self.record_button = ctk.CTkButton(            record_controls,            text="🎤 Start Recording",            command=self.toggle_recording,            height=40,            fg_color=("gray75", "gray25")        )        self.record_button.pack(fill="x", pady=(0, 5))                # Recording status        self.recording_status = ctk.CTkLabel(            record_controls,            text="Ready to record",            font=ctk.CTkFont(size=11)        )        self.recording_status.pack(anchor="w", pady=(0, 5))                # Recording timer        self.recording_timer = ctk.CTkLabel(            record_controls,            text="",            font=ctk.CTkFont(size=11),            text_color=("red", "lightcoral")        )        self.recording_timer.pack(anchor="w")                # Use recording button        self.use_recording_button = ctk.CTkButton(            record_controls,            text="📂 Use Recording",            command=self.use_recording,            height=32,            state="disabled"        )        self.use_recording_button.pack(fill="x", pady=(5, 0))                # Step 2: Transcription Section        step2_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")        step2_frame.pack(fill="x", padx=20, pady=(0, 15))                ctk.CTkLabel(            step2_frame,            text="Step 2: Transcribe Audio",            font=ctk.CTkFont(size=14, weight="bold")        ).pack(anchor="w", pady=(0, 10))                # Whisper Model Selection        model_container = ctk.CTkFrame(step2_frame)        model_container.pack(fill="x", pady=(0, 10))                ctk.CTkLabel(model_container, text="Whisper Model:", font=ctk.CTkFont(size=12)).pack(anchor="w", pady=(5, 5))                model_select_frame = ctk.CTkFrame(model_container)        model_select_frame.pack(fill="x")                self.model_var = ctk.StringVar(value="base")        models = ["tiny", "base", "small", "medium", "large"]        self.model_menu = ctk.CTkOptionMenu(            model_select_frame,            values=models,            variable=self.model_var,            command=self.on_model_change,            width=140        )        self.model_menu.pack(side="left", pady=(0, 5))                # Model status indicator inline        self.model_status = ctk.CTkLabel(            model_select_frame,             text="⚪ Not loaded",            font=ctk.CTkFont(size=11)        )        self.model_status.pack(side="left", padx=(10, 0))                # Transcribe Button        self.transcribe_button = ctk.CTkButton(            step2_frame,            text="🎙️ Transcribe",            command=self.start_transcription,            height=40,            font=ctk.CTkFont(size=13, weight="bold"),            state="disabled",            fg_color=("gray75", "gray25")        )        self.transcribe_button.pack(fill="x", pady=(5, 0))                # Clean Text Button (NEW)        self.clean_button = ctk.CTkButton(            step2_frame,            text="🧹 Clean Text",            command=self.clean_transcribed_text,            height=35,            state="disabled",            fg_color=("gray75", "gray25")        )        self.clean_button.pack(fill="x", pady=(5, 0))                # Network Plot Button (NEW)        self.network_button = ctk.CTkButton(            step2_frame,            text="🕸️ Generate Network Plot",            command=self.create_network_plot,            height=35,            state="disabled",            fg_color=("gray75", "gray25")        )        self.network_button.pack(fill="x", pady=(5, 0))                # Step 3: Processing Section        step3_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")        step3_frame.pack(fill="x", padx=20, pady=(0, 15))                ctk.CTkLabel(            step3_frame,            text="Step 3: Process with AI",            font=ctk.CTkFont(size=14, weight="bold")        ).pack(anchor="w", pady=(0, 10))                # API Key Entry        api_container = ctk.CTkFrame(step3_frame)        api_container.pack(fill="x", pady=(0, 10))                ctk.CTkLabel(api_container, text="Anthropic API Key:", font=ctk.CTkFont(size=12)).pack(anchor="w", pady=(5, 5))                api_entry_frame = ctk.CTkFrame(api_container)        api_entry_frame.pack(fill="x")                self.api_key_entry = ctk.CTkEntry(api_entry_frame, show="*", width=180)        self.api_key_entry.pack(side="left", pady=(0, 5))                self.save_api_button = ctk.CTkButton(            api_entry_frame,            text="Save",            command=self.save_api_key,            width=60,            height=28        )        self.save_api_button.pack(side="left", padx=(5, 0))                # Summarize Options        summarize_frame = ctk.CTkFrame(step3_frame)        summarize_frame.pack(fill="x", pady=(10, 0))                # Word limit setting        word_limit_frame = ctk.CTkFrame(summarize_frame)        word_limit_frame.pack(fill="x", pady=(0, 10))                ctk.CTkLabel(word_limit_frame, text="Word Limit:", font=ctk.CTkFont(size=12)).pack(side="left", padx=(0, 10))                self.word_limit_var = ctk.StringVar(value="500")        self.word_limit_entry = ctk.CTkEntry(            word_limit_frame,            width=80,            textvariable=self.word_limit_var,            placeholder_text="500"        )        self.word_limit_entry.pack(side="left")                ctk.CTkLabel(word_limit_frame, text="words", font=ctk.CTkFont(size=11)).pack(side="left", padx=(5, 0))                # Summarize Button        self.summarize_button = ctk.CTkButton(            summarize_frame,            text="📝 Summarize",            command=self.summarize_transcription,            height=40,            state="disabled",            font=ctk.CTkFont(size=13, weight="bold")        )        self.summarize_button.pack(fill="x", pady=(5, 0))                # Output Format        format_frame = ctk.CTkFrame(step3_frame)        format_frame.pack(fill="x", pady=(10, 0))                ctk.CTkLabel(format_frame, text="Output Format:", font=ctk.CTkFont(size=11)).pack(side="left", padx=(0, 10))                self.output_format = ctk.StringVar(value="Markdown")        formats = ["Markdown", "Plain Text", "JSON", "Bullet Points", "LaTeX PDF"]        self.format_menu = ctk.CTkOptionMenu(            format_frame,            values=formats,            variable=self.output_format,            width=140,            height=28        )        self.format_menu.pack(side="left", fill="x", expand=True)                # Progress Section (now inside scrollable area)        progress_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")        progress_frame.pack(fill="x", padx=20, pady=(20, 20))                self.progress_bar = ctk.CTkProgressBar(progress_frame)        self.progress_bar.pack(fill="x", pady=(0, 5))        self.progress_bar.set(0)                self.status_label = ctk.CTkLabel(            progress_frame,             text="Ready",             font=ctk.CTkFont(size=11)        )        self.status_label.pack()            def create_main_panel(self):        # Main Panel        self.main_panel = ctk.CTkFrame(self.main_container)        self.main_panel.grid(row=0, column=1, sticky="nsew")        self.main_panel.grid_columnconfigure(0, weight=1)        self.main_panel.grid_rowconfigure(0, weight=1)                # Tab View        self.tabview = ctk.CTkTabview(self.main_panel)        self.tabview.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)                # Transcription Tab        self.trans_tab = self.tabview.add("📝 Transcription")        trans_frame = ctk.CTkFrame(self.trans_tab)        trans_frame.pack(fill="both", expand=True, padx=5, pady=5)                # Transcription header with word count        trans_header = ctk.CTkFrame(trans_frame, height=30)        trans_header.pack(fill="x", pady=(0, 5))                self.trans_word_count = ctk.CTkLabel(            trans_header,             text="Words: 0 | Characters: 0",            font=ctk.CTkFont(size=11)        )        self.trans_word_count.pack(side="left")                self.trans_status = ctk.CTkLabel(            trans_header,            text="No transcription yet",            font=ctk.CTkFont(size=11),            text_color=("gray50", "gray50")        )        self.trans_status.pack(side="right")                self.trans_text = ctk.CTkTextbox(            trans_frame,            font=ctk.CTkFont(size=13),            wrap="word"        )        self.trans_text.pack(fill="both", expand=True)                # Processed Result Tab - Split view        self.result_tab = self.tabview.add("✨ Processed Result")        result_container = ctk.CTkFrame(self.result_tab)        result_container.pack(fill="both", expand=True, padx=5, pady=5)        result_container.grid_columnconfigure(0, weight=1)        result_container.grid_rowconfigure(0, weight=2)  # Upper section gets more space        result_container.grid_rowconfigure(1, weight=1)  # Lower section for custom prompt                # Upper section - API output        upper_frame = ctk.CTkFrame(result_container)        upper_frame.grid(row=0, column=0, sticky="nsew", pady=(0, 5))                # Result header with navigation        result_header = ctk.CTkFrame(upper_frame, height=30)        result_header.pack(fill="x", pady=(0, 5))                # Navigation buttons for history        nav_frame = ctk.CTkFrame(result_header)        nav_frame.pack(side="left")                self.prev_button = ctk.CTkButton(            nav_frame,            text="◀",            width=30,            height=25,            command=self.navigate_history_prev,            state="disabled"        )        self.prev_button.pack(side="left", padx=2)                self.history_label = ctk.CTkLabel(            nav_frame,            text="0/0",            font=ctk.CTkFont(size=11),            width=50        )        self.history_label.pack(side="left", padx=5)                self.next_button = ctk.CTkButton(            nav_frame,            text="▶",            width=30,            height=25,            command=self.navigate_history_next,            state="disabled"        )        self.next_button.pack(side="left", padx=2)                self.result_status = ctk.CTkLabel(            result_header,            text="No processed result yet",            font=ctk.CTkFont(size=11),            text_color=("gray50", "gray50")        )        self.result_status.pack(side="right")                self.result_text = ctk.CTkTextbox(            upper_frame,            font=ctk.CTkFont(size=13),            wrap="word"        )        self.result_text.pack(fill="both", expand=True)                # Lower section - Custom prompt        lower_frame = ctk.CTkFrame(result_container)        lower_frame.grid(row=1, column=0, sticky="nsew", pady=(5, 0))                # Custom prompt header        prompt_header = ctk.CTkFrame(lower_frame)        prompt_header.pack(fill="x", pady=(5, 5))                ctk.CTkLabel(            prompt_header,            text="Custom Prompt (use {text} for transcription):",            font=ctk.CTkFont(size=12)        ).pack(side="left")                # Custom prompt input        prompt_input_frame = ctk.CTkFrame(lower_frame)        prompt_input_frame.pack(fill="both", expand=True, pady=(0, 5))                self.custom_prompt_text = ctk.CTkTextbox(            prompt_input_frame,            height=100,            font=ctk.CTkFont(size=12)        )        self.custom_prompt_text.pack(fill="both", expand=True, pady=(0, 5))        self.custom_prompt_text.insert("1.0", "Analyze the following text and provide insights:\n\n{text}")                # Process custom prompt button        self.process_custom_button = ctk.CTkButton(            prompt_input_frame,            text="▶ Process Custom Prompt",            command=self.process_custom_prompt,            height=32,            state="disabled"        )        self.process_custom_button.pack(fill="x")                # Bottom toolbar        toolbar_frame = ctk.CTkFrame(self.main_panel)        toolbar_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=(0, 10))                # Left side buttons        left_buttons = ctk.CTkFrame(toolbar_frame)        left_buttons.pack(side="left")                ctk.CTkButton(            left_buttons,            text="📋 Copy Current Tab",            command=self.copy_to_clipboard,            width=130,            height=32        ).pack(side="left", padx=2)                ctk.CTkButton(            left_buttons,            text="💾 Export Transcription",            command=lambda: self.export_text("transcription"),            width=140,            height=32        ).pack(side="left", padx=2)                ctk.CTkButton(            left_buttons,            text="💾 Export Result",            command=lambda: self.export_text("result"),            width=120,            height=32        ).pack(side="left", padx=2)                # Right side buttons        ctk.CTkButton(            toolbar_frame,            text="🗑️ Clear All",            command=self.clear_all,            width=100,            height=32,            fg_color=("gray75", "gray25")        ).pack(side="right", padx=2)        def navigate_history_prev(self):        """Navigate to previous result in history"""        if self.current_history_index > 0:            self.current_history_index -= 1            self.display_history_item()        def navigate_history_next(self):        """Navigate to next result in history"""        if self.current_history_index < len(self.process_history) - 1:            self.current_history_index += 1            self.display_history_item()        def display_history_item(self):        """Display the current history item"""        if 0 <= self.current_history_index < len(self.process_history):            prompt, result = self.process_history[self.current_history_index]                        # Update result text            self.result_text.delete("1.0", "end")            self.result_text.insert("1.0", result)                        # Update custom prompt text            self.custom_prompt_text.delete("1.0", "end")            self.custom_prompt_text.insert("1.0", prompt)                        # Update navigation            self.update_history_navigation()        def update_history_navigation(self):        """Update history navigation buttons and label"""        total = len(self.process_history)        current = self.current_history_index + 1 if total > 0 else 0                self.history_label.configure(text=f"{current}/{total}")                # Update button states        self.prev_button.configure(state="normal" if self.current_history_index > 0 else "disabled")        self.next_button.configure(state="normal" if self.current_history_index < total - 1 else "disabled")        def add_to_history(self, prompt: str, result: str):        """Add a new result to history"""        self.process_history.append((prompt, result))        self.current_history_index = len(self.process_history) - 1        self.update_history_navigation()        def check_error_queue(self):        """Check for errors from background threads"""        try:            while True:                error_type, error_msg = self.error_queue.get_nowait()                messagebox.showerror(error_type, error_msg)        except queue.Empty:            pass        finally:            self.after(100, self.check_error_queue)        def save_api_key(self):        """Save API key for session"""        api_key = self.api_key_entry.get()        if api_key:            self.api_key = api_key            self.save_api_button.configure(text="✓")            self.after(2000, lambda: self.save_api_button.configure(text="Save"))            self.update_status("API key saved")            self.update_button_states()  # Add this line        def copy_to_clipboard(self):        """Copy current tab content to clipboard"""        current_tab = self.tabview.get()        if "Transcription" in current_tab and self.transcribed_text:            self.clipboard_clear()            self.clipboard_append(self.transcribed_text)            self.update_status("Transcription copied to clipboard")        elif "Result" in current_tab and self.current_history_index >= 0:            _, result = self.process_history[self.current_history_index]            self.clipboard_clear()            self.clipboard_append(result)            self.update_status("Result copied to clipboard")        else:            self.update_status("No content to copy")        def load_whisper_model(self):        """Load Whisper model in background thread"""        if self.model_loading:            return                    def load():            self.model_loading = True            self.after(0, lambda: self.update_status("Loading Whisper model..."))            self.after(0, lambda: self.model_status.configure(text="🟡 Loading..."))                        try:                model_name = self.model_var.get()                self.whisper_model = whisper.load_model(model_name)                                self.after(0, lambda: self.update_status("Whisper model loaded"))                self.after(0, lambda: self.model_status.configure(text="🟢 Ready"))                self.after(0, lambda: self.update_button_states())                            except Exception as e:                self.error_queue.put(("Model Load Error", f"Failed to load Whisper model: {str(e)}"))                self.after(0, lambda: self.model_status.configure(text="🔴 Error"))            finally:                self.model_loading = False                threading.Thread(target=load, daemon=True).start()        def on_model_change(self, value):        """Handle model selection change"""        self.whisper_model = None        self.model_status.configure(text="⚪ Not loaded")        self.update_button_states()        self.load_whisper_model()        def select_audio_file(self):        """Open file dialog to select audio/video file"""        file_types = [            ("Audio/Video files", "*.mp3 *.wav *.m4a *.flac *.aac *.ogg *.wma *.mp4 *.mov *.avi *.mkv *.webm *.m4v"),            ("Audio files", "*.mp3 *.wav *.m4a *.flac *.aac *.ogg *.wma"),            ("Video files", "*.mp4 *.mov *.avi *.mkv *.webm *.m4v"),            ("MP3 files", "*.mp3"),            ("MP4 files", "*.mp4"),            ("WAV files", "*.wav"),            ("All files", "*.*")        ]                file_path = filedialog.askopenfilename(            title="Select Audio/Video File",            filetypes=file_types        )                if file_path:            self.audio_file_path = file_path            filename = os.path.basename(file_path)            self.file_label.configure(text=f"Selected: {filename}")                        # Reset transcription when new file is loaded            self.transcribed_text = ""            self.trans_text.delete("1.0", "end")            self.trans_word_count.configure(text="Words: 0 | Characters: 0")            self.trans_status.configure(text="Ready to transcribe", text_color=("gray50", "gray50"))                        # Update button states            self.update_button_states()                        # Determine file type for status message            file_ext = os.path.splitext(file_path)[1].lower()            video_extensions = {'.mp4', '.mov', '.avi', '.mkv', '.webm', '.m4v'}            file_type = "video" if file_ext in video_extensions else "audio"            self.update_status(f"{file_type.capitalize()} file selected: {filename}")        def update_button_states(self):        """Update all button states based on current conditions"""                # Get current API key        current_api_key = self.api_key if self.api_key else self.api_key_entry.get()                # Transcribe button - enable if we have a file AND model is loaded        if self.audio_file_path and self.whisper_model:            self.transcribe_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))        else:            self.transcribe_button.configure(state="disabled", fg_color=("gray75", "gray25"))                # Clean button - enable if we have transcribed text AND API key        if self.transcribed_text and current_api_key:            self.clean_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))        else:            self.clean_button.configure(state="disabled", fg_color=("gray75", "gray25"))                # Network button - enable if we have transcribed text AND API key        if self.transcribed_text and current_api_key:            self.network_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))        else:            self.network_button.configure(state="disabled", fg_color=("gray75", "gray25"))                # Summarize button - enable if we have transcribed text AND API key        if self.transcribed_text and current_api_key:            self.summarize_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))        else:            self.summarize_button.configure(state="disabled", fg_color=("gray75", "gray25"))                # Custom prompt button - enable if we have transcribed text AND API key        if self.transcribed_text and current_api_key:            self.process_custom_button.configure(state="normal")        else:            self.process_custom_button.configure(state="disabled")        def update_status(self, message: str):        """Update status label"""        self.status_label.configure(text=message)        def start_transcription(self):        """Start transcription in background thread"""        if not self.audio_file_path or not self.whisper_model:            return                def transcribe():            try:                # Determine file type for status message                file_ext = os.path.splitext(self.audio_file_path)[1].lower()                video_extensions = {'.mp4', '.mov', '.avi', '.mkv', '.webm', '.m4v'}                file_type = "video" if file_ext in video_extensions else "audio"                                self.after(0, lambda: self.update_status(f"Transcribing {file_type}..."))                self.after(0, lambda: self.progress_bar.set(0.1))                self.after(0, lambda: self.transcribe_button.configure(state="disabled"))                                # Transcribe audio (Whisper automatically extracts audio from video files)                result = self.whisper_model.transcribe(self.audio_file_path)                transcribed_text = result["text"]                                # Update UI on main thread                self.after(0, lambda: self.update_transcription_result(transcribed_text))                            except Exception as e:                self.error_queue.put(("Transcription Error", f"Failed to transcribe {file_type}: {str(e)}"))            finally:                self.after(0, lambda: self.progress_bar.set(0))                self.after(0, lambda: self.update_button_states())                threading.Thread(target=transcribe, daemon=True).start()        def update_transcription_result(self, text: str):        """Update transcription result in UI"""        self.transcribed_text = text        self.trans_text.delete("1.0", "end")        self.trans_text.insert("1.0", text)                # Update word count        words = len(text.split())        chars = len(text)        self.trans_word_count.configure(text=f"Words: {words} | Characters: {chars}")        self.trans_status.configure(text="Transcription complete", text_color=("green", "lightgreen"))                self.update_status("Transcription completed")        self.progress_bar.set(1.0)        self.update_button_states()        def summarize_transcription(self):        """Summarize transcription using Claude"""        if not self.transcribed_text:            return                try:            word_limit = int(self.word_limit_var.get() or "500")        except ValueError:            word_limit = 500                prompt = self.get_summarize_prompt(self.transcribed_text, word_limit)                def process():            try:                self.after(0, lambda: self.summarize_button.configure(state="disabled"))                                result = self.process_with_claude(self.transcribed_text, prompt)                                if result:                    self.after(0, lambda: self.update_result(prompt, result))                    self.after(0, lambda: self.update_status("Summarization completed"))                else:                    self.after(0, lambda: self.update_status("Summarization failed"))                            except Exception as e:                self.error_queue.put(("Processing Error", f"Failed to process: {str(e)}"))            finally:                self.after(0, lambda: self.progress_bar.set(0))                self.after(0, lambda: self.update_button_states())                threading.Thread(target=process, daemon=True).start()        def process_custom_prompt(self):        """Process custom prompt with Claude"""        if not self.transcribed_text:            return                custom_prompt = self.custom_prompt_text.get("1.0", "end-1c")                # Replace {text} placeholder with actual transcription        prompt = custom_prompt.replace("{text}", self.transcribed_text)                def process():            try:                self.after(0, lambda: self.process_custom_button.configure(state="disabled"))                                result = self.process_with_claude(self.transcribed_text, prompt)                                if result:                    self.after(0, lambda: self.update_result(custom_prompt, result))                    self.after(0, lambda: self.update_status("Custom prompt processed"))                else:                    self.after(0, lambda: self.update_status("Processing failed"))                            except Exception as e:                self.error_queue.put(("Processing Error", f"Failed to process: {str(e)}"))            finally:                self.after(0, lambda: self.progress_bar.set(0))                self.after(0, lambda: self.update_button_states())                threading.Thread(target=process, daemon=True).start()        def update_result(self, prompt: str, result: str):        """Update result in UI and add to history"""        self.add_to_history(prompt, result)                self.result_text.delete("1.0", "end")        self.result_text.insert("1.0", result)                self.result_status.configure(text="Processing complete", text_color=("green", "lightgreen"))                # Switch to result tab        self.tabview.set("✨ Processed Result")        def get_summarize_prompt(self, text: str, word_limit: int) -> str:        """Generate summarization prompt with word limit"""        output_format = self.output_format.get().lower()                format_instructions = {            "markdown": "Format your response in clean Markdown with appropriate headers and structure.",            "plain text": "Provide a plain text response without any formatting.",            "json": "Return your response as a well-structured JSON object.",            "bullet points": "Structure your response as clear bullet points.",            "latex pdf": "Format your response in LaTeX document format with proper document class, sections, and formatting."        }                prompt = f"""Please provide a comprehensive summary of the following transcribed audio content in approximately {word_limit} words.         Focus on the main ideas, key points, and important details.         {format_instructions.get(output_format, '')}                Transcribed text:        {text}"""                return prompt        def process_with_claude(self, text: str, prompt: str) -> Optional[str]:        """Process text with Claude API"""        api_key = self.api_key_entry.get() or self.api_key                try:            self.after(0, lambda: self.update_status("Calling Claude API..."))            self.after(0, lambda: self.progress_bar.set(0.7))                        client = Anthropic(api_key=api_key)                        # Add instructions for math formatting if LaTeX output is selected            if self.output_format.get() == "LaTeX PDF":                prompt += "\n\nIMPORTANT: Format all mathematical equations using proper LaTeX syntax. Use equation environments for display equations and inline math mode for inline expressions. Ensure all special symbols are properly escaped."                        response = client.messages.create(                model="claude-sonnet-4-20250514",                max_tokens=4000,                temperature=0.3,                messages=[{"role": "user", "content": prompt}]            )                        result_text = response.content[0].text                        # Handle LaTeX PDF output if selected            if self.output_format.get() == "LaTeX PDF":                result_text = self.ensure_latex_format(result_text)                        self.after(0, lambda: self.progress_bar.set(1.0))            return result_text                    except Exception as e:            self.error_queue.put(("Claude API Error", f"Failed to process: {str(e)}"))            return None        def ensure_latex_format(self, text: str) -> str:        """Ensure text is in proper LaTeX format with beautiful math rendering"""                # Clean up problematic characters        text = text.replace('\u2019', "'")        text = text.replace('\u201c', '``')        text = text.replace('\u201d', "''")        text = text.replace('\u2013', '--')        text = text.replace('\u2014', '---')                # Convert markdown-style math to LaTeX if present        import re                # Convert inline code blocks with math to LaTeX math mode        text = re.sub(r'```\n?(.*?)\n?```', r'\\[\n\1\n\\]', text, flags=re.DOTALL)        text = re.sub(r'`([^`]+)`', r'$\1$', text)                # Convert markdown headers to LaTeX sections        text = re.sub(r'^### (.*?)$', r'\\subsection{\1}', text, flags=re.MULTILINE)        text = re.sub(r'^## (.*?)$', r'\\section{\1}', text, flags=re.MULTILINE)        text = re.sub(r'^# (.*?)$', r'\\chapter{\1}', text, flags=re.MULTILINE)                # Convert numbered items to subsections        text = re.sub(r'^(\d+)\.\s*([A-Z].*?)$', r'\\subsection*{\\textbf{\1. \2}}', text, flags=re.MULTILINE)                # Convert bullet points        text = re.sub(r'^- (.*?)$', r'\\item \1', text, flags=re.MULTILINE)                # Handle "Where:" sections        if 'Where:' in text:            lines = text.split('\n')            new_lines = []            in_where = False                        for line in lines:                if 'Where:' in line:                    new_lines.append(line.replace('Where:', '\\textbf{Where:}'))                    new_lines.append('\\begin{itemize}')                    in_where = True                elif in_where and line.strip() and not line.strip().startswith('\\item'):                    if line.strip().startswith('-'):                        new_lines.append(line)                    elif re.match(r'^\d+\.', line.strip()) or line.strip().startswith('\\'):                        new_lines.append('\\end{itemize}')                        new_lines.append(line)                        in_where = False                    else:                        new_lines.append(line)                else:                    new_lines.append(line)                        if in_where:                new_lines.append('\\end{itemize}')                        text = '\n'.join(new_lines)                # Process mathematical equations        def format_equations(text):            """Format equations for proper LaTeX display"""            lines = text.split('\n')            formatted_lines = []                        for line in lines:                # Check if line contains an equation pattern                if '=' in line and any(char in line for char in ['(', ')', '²', '·', 'Σ', '∂', 'φ', 'τ', 'ε']):                    equation = line.strip()                                        # Replace special characters with LaTeX commands                    replacements = [                        ('²', '^2'),                        ('·', ' \\cdot '),                        ('Σ', '\\sum'),                        ('∂', '\\partial'),                        ('φ', '\\phi'),                        ('Δφ', '\\Delta\\phi'),                        ('τ', '\\tau'),                        ('ε', '\\varepsilon'),                        ('σ', '\\sigma'),                        ('→', '\\rightarrow'),                        ('√', '\\sqrt'),                    ]                                        for old, new in replacements:                        equation = equation.replace(old, new)                                        # Format simple fractions                    equation = re.sub(r'(\w+)\s*/\s*(\w+)', r'\\frac{\1}{\2}', equation)                                        # Wrap in equation environment if not already wrapped                    if not equation.startswith('\\[') and not equation.startswith('$$'):                        formatted_lines.append('\\begin{equation}')                        formatted_lines.append(equation)                        formatted_lines.append('\\end{equation}')                    else:                        formatted_lines.append(equation)                else:                    formatted_lines.append(line)                        return '\n'.join(formatted_lines)                # Apply equation formatting        text = format_equations(text)                if not text.startswith("\\documentclass"):            # Create a complete LaTeX document            latex_text = r"""\documentclass[11pt]{article}    \usepackage[utf8]{inputenc}    \usepackage[T1]{fontenc}    \usepackage{lmodern}    \usepackage{amsmath}    \usepackage{amssymb}    \usepackage{amsfonts}    \usepackage{amsthm}    \usepackage{mathtools}    \usepackage{geometry}    \usepackage{enumitem}    \usepackage{hyperref}    \geometry{a4paper, margin=1in}        \title{Audio Transcription Analysis}    \author{Generated Analysis}    \date{\today}        \begin{document}    \maketitle        """ + text + r"""        \end{document}"""            return latex_text        else:            # Document already has structure            if "\\usepackage{amsmath}" not in text:                lines = text.split('\n')                for i, line in enumerate(lines):                    if line.startswith("\\documentclass"):                        math_packages = [                            "\\usepackage[utf8]{inputenc}",                            "\\usepackage[T1]{fontenc}",                            "\\usepackage{amsmath}",                            "\\usepackage{amssymb}",                            "\\usepackage{amsfonts}",                        ]                        for j, pkg in enumerate(math_packages):                            if pkg not in text:                                lines.insert(i + 1 + j, pkg)                        break                text = '\n'.join(lines)                return text        def export_text(self, text_type: str):        """Export transcription or result to file with proper encoding"""        if text_type == "transcription" and not self.transcribed_text:            messagebox.showwarning("No Content", "No transcription to export")            return        elif text_type == "result" and (self.current_history_index < 0 or not self.process_history):            messagebox.showwarning("No Content", "No processed result to export")            return                output_format = self.output_format.get()                if output_format == "LaTeX PDF":            # For LaTeX PDF, we need to compile to PDF            self.export_as_latex_pdf(text_type)  # Modified to support both types        else:            # Regular text export            default_ext = self.get_default_extension()                        file_path = filedialog.asksaveasfilename(                defaultextension=default_ext,                filetypes=[                    ("Text Files", "*.txt"),                    ("Markdown Files", "*.md"),                    ("JSON Files", "*.json"),                    ("LaTeX Files", "*.tex"),                    ("All Files", "*.*")                ]            )                        if file_path:                if text_type == "transcription":                    content = self.transcribed_text                else:                    _, content = self.process_history[self.current_history_index]                                try:                    # Use UTF-8 encoding with error handling                    with open(file_path, 'w', encoding='utf-8', errors='replace') as f:                        # Clean problematic characters before saving                        content = self.clean_text_for_export(content)                        f.write(content)                    messagebox.showinfo("Export Success", f"File saved successfully")                except Exception as e:                    # Fallback to latin-1 encoding if UTF-8 fails                    try:                        with open(file_path, 'w', encoding='latin-1', errors='replace') as f:                            f.write(content)                        messagebox.showinfo("Export Success", f"File saved successfully (using latin-1 encoding)")                    except Exception as e2:                        messagebox.showerror("Export Error", f"Failed to save file: {str(e2)}")        def get_default_extension(self) -> str:        """Get default file extension based on output format"""        format_map = {            "Markdown": ".md",            "Plain Text": ".txt",            "JSON": ".json",            "Bullet Points": ".txt",            "LaTeX PDF": ".tex"        }        return format_map.get(self.output_format.get(), ".txt")        def export_as_latex_pdf(self, text_type="result"):        """Export result or transcription as compiled LaTeX PDF"""        if text_type == "transcription":            if not self.transcribed_text:                messagebox.showwarning("No Content", "No transcription to export")                return            content = self.transcribed_text        else:            if self.current_history_index < 0 or not self.process_history:                messagebox.showwarning("No Content", "No processed result to export")                return            _, content = self.process_history[self.current_history_index]                # Ensure content is in LaTeX format        latex_content = self.ensure_latex_format(content)                # Ask user where to save PDF        pdf_path = filedialog.asksaveasfilename(            defaultextension=".pdf",            filetypes=[("PDF Files", "*.pdf"), ("All Files", "*.*")]        )                if not pdf_path:            return                try:            # Create temporary directory for LaTeX compilation            with tempfile.TemporaryDirectory() as temp_dir:                tex_file = Path(temp_dir) / "document.tex"                                # Write LaTeX content with proper encoding handling                with open(tex_file, 'w', encoding='utf-8', errors='replace') as f:                    f.write(latex_content)                                # Try to compile with pdflatex                try:                    # Try different possible paths for pdflatex on macOS                    pdflatex_paths = [                        "/usr/local/texlive/2025/bin/universal-darwin/pdflatex",                        "/usr/local/texlive/2024/bin/universal-darwin/pdflatex",                        "/usr/local/texlive/2023/bin/universal-darwin/pdflatex",                        "/Library/TeX/texbin/pdflatex",                        "pdflatex",                    ]                                        pdflatex_cmd = None                    for path in pdflatex_paths:                        if os.path.exists(path) or path == "pdflatex":                            try:                                subprocess.run([path, "--version"], capture_output=True, timeout=2)                                pdflatex_cmd = path                                break                            except:                                continue                                        if not pdflatex_cmd:                        raise FileNotFoundError("pdflatex not found")                                        # Run pdflatex with proper encoding                    env = os.environ.copy()                    env['LANG'] = 'en_US.UTF-8'                                        for _ in range(2):                        result = subprocess.run(                            [pdflatex_cmd, "-interaction=nonstopmode", "-output-directory", temp_dir, str(tex_file)],                            capture_output=True,                            text=True,                            timeout=30,                            env=env,                            encoding='utf-8',                            errors='replace'                        )                                        # Check if PDF was created                    pdf_file = Path(temp_dir) / "document.pdf"                    if pdf_file.exists():                        # Copy to destination                        import shutil                        shutil.copy(pdf_file, pdf_path)                        messagebox.showinfo("Export Success", "PDF exported successfully")                    else:                        # Check for compilation errors                        log_file = Path(temp_dir) / "document.log"                        if log_file.exists():                            with open(log_file, 'r', encoding='utf-8', errors='replace') as f:                                log_content = f.read()                            if "Error" in log_content:                                raise Exception("LaTeX compilation failed. Check your LaTeX syntax.")                        raise Exception("PDF file was not generated")                                            except FileNotFoundError:                    # pdflatex not installed or not found                    messagebox.showerror(                        "LaTeX Not Found",                        "pdflatex could not be found.\n\n"                        "Please install MacTeX or TeX Live.\n\n"                        "Alternatively, you can export as .tex and compile manually."                    )                                        # Offer to save as .tex instead                    if messagebox.askyesno("Save as LaTeX", "Would you like to save as a .tex file instead?"):                        tex_path = pdf_path.replace('.pdf', '.tex')                        with open(tex_path, 'w', encoding='utf-8', errors='replace') as f:                            f.write(latex_content)                        messagebox.showinfo("Export Success", f"LaTeX file saved to {tex_path}")                                    except Exception as e:            messagebox.showerror("Export Error", f"Failed to export PDF: {str(e)}")    def clean_text_for_export(self, text):        """Clean problematic characters for export"""        replacements = [            ('\u2019', "'"),            ('\u201c', '"'),            ('\u201d', '"'),            ('\u2013', '-'),            ('\u2014', '--'),            ('\u2026', '...'),            ('\u00a0', ' '),            ('\u200b', ''),        ]        for old, new in replacements:            text = text.replace(old, new)        return text    def clean_transcribed_text(self):        """Clean transcribed text using Claude API"""        if not self.transcribed_text:            messagebox.showwarning("No Content", "No transcription to clean")            return                api_key = self.api_key_entry.get() or self.api_key        if not api_key:            messagebox.showwarning("API Key Required", "Please enter your Anthropic API key")            return                def clean():            try:                self.after(0, lambda: self.clean_button.configure(state="disabled"))                self.after(0, lambda: self.update_status("Cleaning transcription..."))                self.after(0, lambda: self.progress_bar.set(0.5))                                client = Anthropic(api_key=api_key)                                prompt = f"""Clean the following transcribed text by:    1. Correcting any obvious transcription errors or strange words    2. Fixing grammar and punctuation while preserving the original meaning    3. Removing filler words (um, uh, etc.) where appropriate    4. Making the text flow naturally as written prose    5. DO NOT summarize or remove content - just clean and correct        Return ONLY the cleaned text without any commentary or explanations.        Text to clean:    {self.transcribed_text}"""                                response = client.messages.create(                    model="claude-sonnet-4-20250514",                    max_tokens=4000,                    temperature=0.2,                    messages=[{"role": "user", "content": prompt}]                )                                cleaned_text = response.content[0].text                                # Update transcription with cleaned text                self.after(0, lambda: self.update_transcription_result(cleaned_text))                self.after(0, lambda: self.trans_status.configure(                    text="Transcription cleaned",                     text_color=("blue", "lightblue")                ))                self.after(0, lambda: self.update_status("Transcription cleaned successfully"))                            except Exception as e:                self.error_queue.put(("Cleaning Error", f"Failed to clean text: {str(e)}"))            finally:                self.after(0, lambda: self.progress_bar.set(0))                self.after(0, lambda: self.clean_button.configure(state="normal"))                threading.Thread(target=clean, daemon=True).start()    def create_network_plot(self):        """Create a network plot using Word2Vec embeddings with user-selectable clustering"""        if not self.transcribed_text:            messagebox.showwarning("No Content", "No transcription available for network plot")            return                api_key = self.api_key_entry.get() or self.api_key        if not api_key:            messagebox.showwarning("API Key Required", "Please enter your Anthropic API key")            return                # Ask user for number of clusters        cluster_dialog = ctk.CTkInputDialog(            text="Enter number of clusters (2-10):",            title="Cluster Configuration"        )        n_clusters_str = cluster_dialog.get_input()                try:            n_clusters = int(n_clusters_str) if n_clusters_str else 5            n_clusters = max(2, min(10, n_clusters))        except:            n_clusters = 5                def generate_plot():            try:                self.after(0, lambda: self.network_button.configure(state="disabled"))                self.after(0, lambda: self.update_status("Loading Word2Vec model..."))                self.after(0, lambda: self.progress_bar.set(0.1))                                # Load or cache Word2Vec model with better error handling                if not hasattr(self, 'word2vec_model'):                    try:                        import gensim.downloader as api                        # Try to load a smaller, faster model first                        self.after(0, lambda: self.update_status("Downloading Word2Vec model (first time only)..."))                        self.word2vec_model = api.load('glove-wiki-gigaword-50')                    except Exception as e:                        self.error_queue.put(("Model Error", f"Failed to load Word2Vec model: {str(e)}\nPlease install: pip install gensim"))                        return                                self.after(0, lambda: self.update_status("Processing text with Word2Vec..."))                self.after(0, lambda: self.progress_bar.set(0.3))                                # Preprocess text                text = self.transcribed_text.lower()                words = re.findall(r'\b[a-z]+\b', text)                                # Enhanced stop words list                stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',                             'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'were', 'be',                             'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',                             'would', 'could', 'should', 'may', 'might', 'must', 'shall', 'can',                             'need', 'dare', 'ought', 'used', 'i', 'you', 'he', 'she', 'it', 'we',                             'they', 'them', 'their', 'this', 'that', 'these', 'those'}                                # Filter words and keep only those in Word2Vec vocabulary                filtered_words = []                for w in words:                    if w not in stop_words and len(w) > 2:                        try:                            # Check if word is in model vocabulary                            if hasattr(self.word2vec_model, 'key_to_index'):                                if w in self.word2vec_model.key_to_index:                                    filtered_words.append(w)                            elif w in self.word2vec_model:                                filtered_words.append(w)                        except:                            pass                                if len(filtered_words) < 5:                    self.error_queue.put(("Insufficient Data", "Not enough words found in the Word2Vec vocabulary. The text may be too short or contain specialized terms."))                    return                                # Get most frequent words that have embeddings                word_freq = Counter(filtered_words)                top_words = [word for word, freq in word_freq.most_common(150)]                                if len(top_words) < 5:                    self.error_queue.put(("Insufficient Data", "Not enough words with embeddings found. Try with longer text."))                    return                                # Get Word2Vec embeddings                embeddings = []                valid_words = []                for word in top_words:                    try:                        if hasattr(self.word2vec_model, 'get_vector'):                            embeddings.append(self.word2vec_model.get_vector(word))                        else:                            embeddings.append(self.word2vec_model[word])                        valid_words.append(word)                    except:                        pass                                if len(embeddings) < 5:                    self.error_queue.put(("Insufficient Embeddings", "Not enough word embeddings found. The text may be too specialized."))                    return                                embeddings = np.array(embeddings)                                self.after(0, lambda: self.update_status("Clustering words in embedding space..."))                self.after(0, lambda: self.progress_bar.set(0.4))                                # Perform clustering in high-dimensional space                from sklearn.cluster import KMeans                n_clusters_actual = min(n_clusters, len(valid_words) // 2)                kmeans = KMeans(n_clusters=n_clusters_actual, random_state=42, n_init=10)                cluster_labels = kmeans.fit_predict(embeddings)                                # Create co-occurrence information for edge weights                window_size = 7                co_occurrences = {}                word_positions = {word: [] for word in valid_words}                                # Record positions of valid words                for i, word in enumerate(filtered_words):                    if word in valid_words:                        word_positions[word].append(i)                                # Calculate co-occurrences with distance weighting                for word1 in valid_words:                    for word2 in valid_words:                        if word1 != word2:                            pair = tuple(sorted([word1, word2]))                            if pair not in co_occurrences:                                co_occurrences[pair] = 0                                for pos1 in word_positions[word1]:                                    for pos2 in word_positions[word2]:                                        distance = abs(pos1 - pos2)                                        if distance <= window_size:                                            # Weight by inverse distance                                            co_occurrences[pair] += 1.0 / (1 + distance)                                self.after(0, lambda: self.update_status("Projecting to 2D space..."))                self.after(0, lambda: self.progress_bar.set(0.5))                                # Project to 2D using t-SNE for better visualization                from sklearn.manifold import TSNE                from sklearn.decomposition import PCA                                if len(embeddings) > 30:                    # Use PCA first for dimensionality reduction if many words                    pca = PCA(n_components=min(30, len(embeddings)-1))                    embeddings_reduced = pca.fit_transform(embeddings)                    tsne = TSNE(n_components=2, perplexity=min(30, len(embeddings)-1),                                random_state=42, n_iter=1000)                    coords_2d = tsne.fit_transform(embeddings_reduced)                else:                    perplexity = min(5, len(embeddings)-1)  # Ensure perplexity is valid                    tsne = TSNE(n_components=2, perplexity=perplexity,                                random_state=42, n_iter=1000)                    coords_2d = tsne.fit_transform(embeddings)                                # Create network graph                import networkx as nx                G = nx.Graph()                                # Add nodes with positions and attributes                for i, word in enumerate(valid_words):                    G.add_node(word,                               pos=(coords_2d[i, 0], coords_2d[i, 1]),                              cluster=cluster_labels[i],                              weight=word_freq[word])                                # Add edges based only on co-occurrence from the transcript                for i, word1 in enumerate(valid_words):                    for j, word2 in enumerate(valid_words[i+1:], i+1):                        pair = tuple(sorted([word1, word2]))                        co_occur = co_occurrences.get(pair, 0)                                                # Add edge only if words co-occur in the transcript                        if co_occur > 0.25:  # Threshold for meaningful co-occurrence                            G.add_edge(word1, word2, weight=co_occur)                                self.after(0, lambda: self.update_status("Creating visualization..."))                self.after(0, lambda: self.progress_bar.set(0.7))                                # Create visualization                import matplotlib.pyplot as plt                fig, ax = plt.subplots(figsize=(14, 10))                fig.patch.set_facecolor('#f0f0f0')                ax.set_facecolor('#ffffff')                                # Use positions from t-SNE (semantic positioning)                pos = nx.get_node_attributes(G, 'pos')                                # Color scheme for clusters                colors = plt.cm.Set3(np.linspace(0, 1, n_clusters_actual))                node_colors = [colors[G.nodes[node]['cluster']] for node in G.nodes()]                                # Node sizes based on word frequency                node_sizes = [300 + G.nodes[node]['weight'] * 30 for node in G.nodes()]                                # Draw edges with varying thickness based on co-occurrence strength                edges = G.edges()                if edges:                    edge_weights = [G[u][v]['weight'] for u, v in edges]                    max_weight = max(edge_weights) if edge_weights else 1                                        # Draw edges with thickness proportional to co-occurrence                    for (u, v), weight in zip(edges, edge_weights):                        alpha = 0.3 + (weight / max_weight) * 0.5                        width = 0.5 + (weight / max_weight) * 3                        nx.draw_networkx_edges(G, pos, [(u, v)], alpha=alpha,                                              width=width, edge_color='#666666', ax=ax)                                # Draw nodes                nx.draw_networkx_nodes(G, pos, node_color=node_colors,                                      node_size=node_sizes, alpha=0.85,                                     edgecolors='black', linewidths=1.5, ax=ax)                                # Draw labels                labels = {node: node for node in G.nodes()}                nx.draw_networkx_labels(G, pos, labels, font_size=9,                                        font_weight='bold', ax=ax)                                # Add title with clarification                ax.set_title("Semantic Network: Spatial Position = Word2Vec Similarity | Edges = Co-occurrence in Transcript",                             fontsize=14, fontweight='bold', pad=20)                ax.axis('off')                                # Add cluster legend                for i in range(n_clusters_actual):                    cluster_words = [w for w in valid_words if cluster_labels[valid_words.index(w)] == i][:3]                    label = f"Cluster {i+1}: {', '.join(cluster_words[:3])}..."                    ax.scatter([], [], c=[colors[i]], s=150, label=label, alpha=0.85)                                ax.legend(loc='upper left', frameon=True, fancybox=True,                          shadow=True, fontsize=9)                                # Add axes labels for dimensions                ax.text(0.02, 0.02, "← Semantic Similarity (Word2Vec) →",                        transform=ax.transAxes, fontsize=9, alpha=0.6)                ax.text(0.02, 0.98, "← Semantic Similarity (Word2Vec) →",                        transform=ax.transAxes, fontsize=9, alpha=0.6, rotation=90, va='top')                                plt.tight_layout()                                # Save plot                plot_path = Path("network_analysis.png")                plt.savefig(plot_path, dpi=150, bbox_inches='tight',                            facecolor='#f0f0f0', edgecolor='none')                plt.close()                                # Show plot in new window                self.after(0, lambda: self.show_network_plot_window(plot_path))                self.after(0, lambda: self.update_status("Network plot generated successfully"))                self.after(0, lambda: self.progress_bar.set(1.0))                            except Exception as e:                self.error_queue.put(("Plot Error", f"Failed to generate network plot: {str(e)}"))            finally:                self.after(0, lambda: self.progress_bar.set(0))                self.after(0, lambda: self.network_button.configure(state="normal"))                threading.Thread(target=generate_plot, daemon=True).start()        # Helper function for simple clustering    def simple_clustering(self, similarity_matrix, n_clusters):        """Simple clustering based on similarity matrix"""        n_items = len(similarity_matrix)        clusters = {i: i for i in range(n_items)}                if n_clusters >= n_items:            return clusters                # Simple hierarchical clustering        while len(set(clusters.values())) > n_clusters:            max_sim = 0            merge_pair = None                        for i in range(n_items):                for j in range(i + 1, n_items):                    if clusters[i] != clusters[j] and similarity_matrix[i][j] > max_sim:                        max_sim = similarity_matrix[i][j]                        merge_pair = (clusters[i], clusters[j])                        if merge_pair:                old_cluster, new_cluster = merge_pair                for i in range(n_items):                    if clusters[i] == old_cluster:                        clusters[i] = new_cluster                # Renumber clusters        unique_clusters = list(set(clusters.values()))        cluster_map = {old: new for new, old in enumerate(unique_clusters)}        return {i: cluster_map[clusters[i]] for i in range(n_items)}        # Function to show network plot in new window    def show_network_plot_window(self, plot_path):        """Display network plot in a new window"""        plot_window = ctk.CTkToplevel(self)        plot_window.title("Semantic Network Analysis")        plot_window.geometry("1000x800")                # Create frame for plot        plot_frame = ctk.CTkFrame(plot_window)        plot_frame.pack(fill="both", expand=True, padx=10, pady=10)                # Load and display image        from PIL import Image, ImageTk        img = Image.open(plot_path)        img = img.resize((950, 750), Image.Resampling.LANCZOS)        photo = ImageTk.PhotoImage(img)                label = ctk.CTkLabel(plot_frame, image=photo, text="")        label.image = photo  # Keep a reference        label.pack()                # Add save button        save_button = ctk.CTkButton(            plot_window,            text="💾 Save Plot",            command=lambda: self.save_network_plot(plot_path),            height=35        )        save_button.pack(pady=10)        # Function to save network plot    def save_network_plot(self, source_path):        """Save network plot to user-specified location"""        file_path = filedialog.asksaveasfilename(            defaultextension=".png",            filetypes=[                ("PNG Files", "*.png"),                ("All Files", "*.*")            ]        )                if file_path:            import shutil            try:                shutil.copy(source_path, file_path)                messagebox.showinfo("Save Success", "Network plot saved successfully")            except Exception as e:                messagebox.showerror("Save Error", f"Failed to save plot: {str(e)}")        def toggle_recording(self):        """Start or stop audio recording"""        if not self.is_recording:            self.start_recording()        else:            self.stop_recording()        def start_recording(self):        """Start recording audio from microphone with fallback options"""        try:            # First try with sounddevice (preferred method)            try:                import sounddevice as sd                devices = sd.query_devices()                input_device = sd.default.device[0]                                if input_device is None:                    raise Exception("No microphone detected with sounddevice")                                # Use sounddevice method                self._start_recording_sounddevice()                return                            except (ImportError, Exception) as e:                print(f"Sounddevice not available: {e}")                                # Fallback to pyaudio if sounddevice fails                try:                    import pyaudio                    self._start_recording_pyaudio()                    return                except ImportError:                    pass                                # Final fallback: use system command (macOS/Linux)                if sys.platform in ['darwin', 'linux']:                    self._start_recording_system()                else:                    messagebox.showerror(                        "Recording Error",                         "Audio recording requires sounddevice or pyaudio.\n"                        "Please install: pip install sounddevice\n"                        "If that fails, try: pip install pyaudio"                    )                            except Exception as e:            messagebox.showerror("Recording Error", f"Failed to start recording: {str(e)}")            self.is_recording = False        def _start_recording_sounddevice(self):        """Recording with sounddevice (original method)"""        import sounddevice as sd                self.is_recording = True        self.recording_data = []        self.recording_start_time = time.time()                # Update UI        self.record_button.configure(            text="⏹️ Stop Recording",            fg_color=("red", "darkred")        )        self.recording_status.configure(text="Recording...")        self.use_recording_button.configure(state="disabled")                # Start recording in background thread        def record_audio():            try:                with sd.InputStream(                    samplerate=self.recording_samplerate,                    channels=1,                    callback=self.audio_callback                ):                    while self.is_recording:                        time.sleep(0.1)                        # Update timer on main thread                        elapsed = time.time() - self.recording_start_time                        mins, secs = divmod(int(elapsed), 60)                        self.after(0, lambda: self.recording_timer.configure(                            text=f"⏱️ {mins:02d}:{secs:02d}"                        ))            except Exception as e:                self.error_queue.put(("Recording Error", str(e)))                self.after(0, self.stop_recording)                self.recording_thread = threading.Thread(target=record_audio, daemon=True)        self.recording_thread.start()        def _start_recording_pyaudio(self):        """Fallback recording with pyaudio"""        import pyaudio                self.is_recording = True        self.recording_data = []        self.recording_start_time = time.time()                # Update UI        self.record_button.configure(            text="⏹️ Stop Recording",            fg_color=("red", "darkred")        )        self.recording_status.configure(text="Recording (pyaudio)...")        self.use_recording_button.configure(state="disabled")                def record_audio():            try:                p = pyaudio.PyAudio()                                stream = p.open(                    format=pyaudio.paInt16,                    channels=1,                    rate=self.recording_samplerate,                    input=True,                    frames_per_buffer=1024                )                                while self.is_recording:                    data = stream.read(1024, exception_on_overflow=False)                    audio_array = np.frombuffer(data, dtype=np.int16).astype(np.float32) / 32768.0                    self.recording_data.append(audio_array.reshape(-1, 1))                                        # Update timer                    elapsed = time.time() - self.recording_start_time                    mins, secs = divmod(int(elapsed), 60)                    self.after(0, lambda: self.recording_timer.configure(                        text=f"⏱️ {mins:02d}:{secs:02d}"                    ))                                stream.stop_stream()                stream.close()                p.terminate()                            except Exception as e:                self.error_queue.put(("Recording Error", str(e)))                self.after(0, self.stop_recording)                self.recording_thread = threading.Thread(target=record_audio, daemon=True)        self.recording_thread.start()        def _start_recording_system(self):        """System command fallback for macOS/Linux"""        self.is_recording = True        self.recording_start_time = time.time()                # Create temporary file for recording        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")        recordings_dir = Path("recordings")        recordings_dir.mkdir(exist_ok=True)        self.recorded_file_path = recordings_dir / f"recording_{timestamp}.wav"                # Update UI        self.record_button.configure(            text="⏹️ Stop Recording",            fg_color=("red", "darkred")        )        self.recording_status.configure(text="Recording (system)...")        self.use_recording_button.configure(state="disabled")                def record_audio():            try:                if sys.platform == 'darwin':  # macOS                    # Use sox or ffmpeg if available                    cmd = [                        'ffmpeg', '-f', 'avfoundation', '-i', ':0',                        '-t', '3600',  # Max 1 hour                        '-ar', str(self.recording_samplerate),                        '-ac', '1',                        str(self.recorded_file_path)                    ]                else:  # Linux                    cmd = [                        'arecord', '-f', 'cd', '-t', 'wav',                        '-d', '3600',  # Max 1 hour                        str(self.recorded_file_path)                    ]                                self.recording_process = subprocess.Popen(                    cmd,                    stdout=subprocess.PIPE,                    stderr=subprocess.PIPE                )                                # Update timer while recording                while self.is_recording and self.recording_process.poll() is None:                    time.sleep(0.1)                    elapsed = time.time() - self.recording_start_time                    mins, secs = divmod(int(elapsed), 60)                    self.after(0, lambda: self.recording_timer.configure(                        text=f"⏱️ {mins:02d}:{secs:02d}"                    ))                            except Exception as e:                self.error_queue.put(("Recording Error", str(e)))                self.after(0, self.stop_recording)                self.recording_thread = threading.Thread(target=record_audio, daemon=True)        self.recording_thread.start()        def audio_callback(self, indata, frames, time_info, status):        """Callback for audio recording"""        if status:            print(f"Recording status: {status}")        self.recording_data.append(indata.copy())        def stop_recording(self):        """Stop recording and save audio file"""        if not self.is_recording:            return                self.is_recording = False                # Update UI        self.record_button.configure(            text="🎤 Start Recording",            fg_color=("gray75", "gray25")        )        self.recording_status.configure(text="Processing recording...")        self.recording_timer.configure(text="")                # Stop system recording if using that method        if hasattr(self, 'recording_process') and self.recording_process is not None:            try:                self.recording_process.terminate()                self.recording_process.wait(timeout=2)            except:                try:                    self.recording_process.kill()                except:                    pass                        # For system recording, file is already saved            if self.recorded_file_path and self.recorded_file_path.exists():                file_size = self.recorded_file_path.stat().st_size                if file_size > 0:                    self.recording_status.configure(                        text=f"Saved: {self.recorded_file_path.name}"                    )                    self.use_recording_button.configure(state="normal")                    self.update_status(f"Recording saved: {self.recorded_file_path.name}")                else:                    self.recording_status.configure(text="Recording failed - empty file")                        # Clean up            self.recording_process = None            return                # Wait for recording thread to finish (for sounddevice/pyaudio methods)        if hasattr(self, 'recording_thread') and self.recording_thread:            self.recording_thread.join(timeout=1)                # Save recording for sounddevice/pyaudio methods        if self.recording_data:            try:                # Combine all audio chunks                audio_data = np.concatenate(self.recording_data, axis=0)                                # Create filename with timestamp                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")                filename = f"recording_{timestamp}.wav"                                # Create recordings directory if it doesn't exist                recordings_dir = Path("recordings")                recordings_dir.mkdir(exist_ok=True)                                # Save audio file                self.recorded_file_path = recordings_dir / filename                                # Try to save with soundfile, fall back to scipy if needed                try:                    import soundfile as sf                    sf.write(                        self.recorded_file_path,                        audio_data,                        self.recording_samplerate                    )                except ImportError:                    try:                        # Fallback to scipy                        from scipy.io import wavfile                        # Convert float32 to int16 for scipy                        audio_int16 = (audio_data * 32767).astype(np.int16)                        wavfile.write(                            self.recorded_file_path,                            self.recording_samplerate,                            audio_int16                        )                    except ImportError:                        # Final fallback: save as numpy array and convert later                        np.save(str(self.recorded_file_path).replace('.wav', '.npy'), audio_data)                        messagebox.showwarning(                            "Format Notice",                             "Recording saved as .npy file. Install soundfile or scipy to save as .wav"                        )                        filename = filename.replace('.wav', '.npy')                        self.recorded_file_path = recordings_dir / filename                                # Update UI                duration = len(audio_data) / self.recording_samplerate                mins, secs = divmod(int(duration), 60)                self.recording_status.configure(                    text=f"Saved: {filename} ({mins:02d}:{secs:02d})"                )                self.use_recording_button.configure(state="normal")                self.update_status(f"Recording saved: {filename}")                            except Exception as e:                messagebox.showerror("Save Error", f"Failed to save recording: {str(e)}")                self.recording_status.configure(text="Failed to save recording")        else:            self.recording_status.configure(text="No audio recorded")                # Clean up        self.recording_data = []        self.recording_thread = None        def use_recording(self):        """Use the last recording as input file"""        if self.recorded_file_path and self.recorded_file_path.exists():            self.audio_file_path = str(self.recorded_file_path)            filename = os.path.basename(self.audio_file_path)            self.file_label.configure(text=f"Selected: {filename}")                        # Reset transcription when new recording is used            self.transcribed_text = ""            self.trans_text.delete("1.0", "end")            self.trans_word_count.configure(text="Words: 0 | Characters: 0")            self.trans_status.configure(text="Ready to transcribe", text_color=("gray50", "gray50"))                        # Update button states to enable transcribe button            self.update_button_states()            self.update_status(f"Using recording: {filename}")                        # Switch to transcription tab            self.tabview.set("📝 Transcription")        else:            messagebox.showwarning("No Recording", "No recording available to use")        def clear_all(self):        """Clear all text content (modified to stop recording if active)"""        # Stop recording if in progress        if self.is_recording:            self.stop_recording()                # Original clear_all code continues...        if self.transcribed_text or self.process_history:            if messagebox.askyesno("Clear All", "Clear all content and history?"):                self.trans_text.delete("1.0", "end")                self.result_text.delete("1.0", "end")                self.transcribed_text = ""                self.process_history = []                self.current_history_index = -1                self.trans_word_count.configure(text="Words: 0 | Characters: 0")                self.trans_status.configure(text="No transcription yet", text_color=("gray50", "gray50"))                self.result_status.configure(text="No processed result yet", text_color=("gray50", "gray50"))                self.update_history_navigation()                self.update_status("Cleared all content")                self.update_button_states()def main():    """Main entry point"""    # Check for required packages    required_packages = {        'whisper': 'openai-whisper',        'anthropic': 'anthropic',        'customtkinter': 'customtkinter',        'sounddevice': 'sounddevice',        'soundfile': 'soundfile'    }        missing_packages = []    for module, package in required_packages.items():        try:            __import__(module)        except ImportError:            missing_packages.append(package)        if missing_packages:        print("Missing required packages. Please install them using:")        print(f"pip install {' '.join(missing_packages)}")        print("\nFor macOS, you may also need:")        print("brew install ffmpeg portaudio")  # Added portaudio        print("\nFor LaTeX PDF export (optional):")        print("- Windows: Install MiKTeX or TeX Live")        print("- macOS: Install MacTeX")        print("- Linux: sudo apt-get install texlive-full")        sys.exit(1)        # Run the application    app = AudioAnalyzerApp()    app.mainloop()if __name__ == "__main__":    main()